### 필수과제3
- sales 데이터를 드릴 예정
- 해당 sales 데이터 중에서 가장 많이 판매되었던 상품을 1, 2 3, 4까지 라벨링해주시고 해당 라벨링 값을 정답으로 하여
- 위에 배운 내용을 복습하는 코드를 작성해주세요!
- 해당 데이터는 정답값이 없다. 우리가 정답값을 만들어야한다.

- 그 정답값을 만들어서 DecisionTree 모델만 사용하여 진행
- skf를 이용해서 동일하게 micro, macro, weighted를 비교하여 정리해주시고
- 데이터의 전처리의 경우 시간이 오래 걸릴 수 있으니 최대한 시간 효율 보셔서 진행해주시면 된다.
- 다만 데이터 전처리가 없으면 안 됩니다.
- 1, 2, 3, 4 외에 나머지 데이터는 0로 해주시고 데이터 전체 삭제하지 마시고 분류해주세요.
- 1, 2, 3, 4, 0 라벨값이 생성될 것

In [1]:
import pandas as pd
import numpy as np
import re

In [120]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score # Confusion matrix 수업 때 진행할 예정
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import classification_report

In [2]:
sales = pd.read_csv("sales_data.csv", encoding="unicode_escape")

In [3]:
sales

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [4]:
# 가장 많이 판매된 상품 상위 4개 확인
sales.Description.value_counts().to_frame()

,Description
WHITE HANGING HEART T-LIGHT HOLDER,2369
REGENCY CAKESTAND 3 TIER,2200
JUMBO BAG RED RETROSPOT,2159
PARTY BUNTING,1727
LUNCH BAG RED RETROSPOT,1638
...,...
Missing,1
historic computer difference?....se,1
DUSTY PINK CHRISTMAS TREE 30CM,1
WRAP BLUE RUSSIAN FOLKART,1


In [5]:
# 이를 바탕으로 레이블 인코딩 진행하여 target 변수를 생성
sales["y"] = 0
sales.loc[sales["Description"] == "WHITE HANGING HEART T-LIGHT HOLDER", "y"] = 1
sales.loc[sales["Description"] == "REGENCY CAKESTAND 3 TIER", "y"] = 2
sales.loc[sales["Description"] == "JUMBO BAG RED RETROSPOT", "y"] = 3
sales.loc[sales["Description"] == "PARTY BUNTING", "y"] = 4

### 데이터 전처리
- 데이터 정보 확인
- 결측치 확인 후 대체

In [6]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
 8   y            541909 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 37.2+ MB


In [7]:
# 결측치 확인
sales.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
y                   0
dtype: int64

- Description 변수의 결측치 행은 분석에서 제외
- CustomerID 열을 전부 제거

In [9]:
# CustomerID 컬럼 제거
sales.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'y'],
      dtype='object')

In [10]:
sales = sales[['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 
               'UnitPrice', 'Country', 'y']]

In [11]:
# Description이 결측치인 행 제거
sales = sales.dropna()
sales.reset_index(inplace=True)

### 상품 분류
- 상품 종류 및 조합을 구분하는 키워드를 중심으로 생성
- packed : 'pack of' 포함 1, o.w. 0
- numbered : 숫자 포함 1, o.w. 0
- set : 'set of' 포함 1, o.w. 0
- box : 'box of' 포함 1, o.w. 0
- amp : '&' 포함 1, o.w. 0
- plus : '+' 포함 1, o.w. 0
- slash : '/' 포함 1, o.w. 0

In [15]:
# Packed 변수 추가
sales["temp"] = sales["Description"].str.extract("(PACK OF)")

In [16]:
label_encoder = LabelEncoder()

label_encoder.fit(sales["temp"])
labels = label_encoder.transform(sales["temp"])
temp = pd.DataFrame(data=labels, columns=["Packed"])
temp["Packed"].replace({0:1, 1:0}, inplace=True)
sales["Packed"] = temp["Packed"]

In [18]:
# Numbered 변수 추가
sales["temp"] = sales["Description"].str.extract("([0-9]+)")
sales["temp"] = sales["temp"].astype("float")
sales.loc[sales["temp"] >= 0, "temp"] = 1

In [19]:
label_encoder.fit(sales["temp"])
labels = label_encoder.transform(sales["temp"])
temp = pd.DataFrame(data=labels, columns=["Numbered"])
temp["Numbered"].replace({0:1, 1:0}, inplace=True)
sales["Numbered"] = temp["Numbered"]

In [20]:
# Set 변수 추가
sales["temp"] = sales["Description"].str.extract("(SET OF)")

In [21]:
label_encoder.fit(sales["temp"])
labels = label_encoder.transform(sales["temp"])
temp = pd.DataFrame(data=labels, columns=["Set"])
temp["Set"].replace({0:1, 1:0}, inplace=True)
sales["Set"] = temp["Set"]

In [22]:
# Box 변수 추가
sales["temp"] = sales["Description"].str.extract("(BOX OF)")

In [23]:
label_encoder.fit(sales["temp"])
labels = label_encoder.transform(sales["temp"])
temp = pd.DataFrame(data=labels, columns=["Box"])
temp["Box"].replace({0:1, 1:0}, inplace=True)
sales["Box"] = temp["Box"]

In [24]:
# Amp 변수 추가
sales["temp"] = sales["Description"].str.extract("(&)")

In [25]:
label_encoder.fit(sales["temp"])
labels = label_encoder.transform(sales["temp"])
temp = pd.DataFrame(data=labels, columns=["Amp"])
temp["Amp"].replace({0:1, 1:0}, inplace=True)
sales["Amp"] = temp["Amp"]

In [26]:
# Plus 변수 추가
sales["temp"] = sales["Description"].str.extract("(\+)")

In [27]:
label_encoder.fit(sales["temp"])
labels = label_encoder.transform(sales["temp"])
temp = pd.DataFrame(data=labels, columns=["Plus"])
temp["Plus"].replace({0:1, 1:0}, inplace=True)
sales["Plus"] = temp["Plus"]

In [53]:
# Slash 변수 추가
sales["temp"] = sales["Description"].str.extract("(\/)")

In [56]:
label_encoder.fit(sales["temp"])
labels = label_encoder.transform(sales["temp"])
temp = pd.DataFrame(data=labels, columns=["Slash"])
temp["Slash"].replace({0:1, 1:0}, inplace=True)
sales["Slash"] = temp["Slash"]

### 상품 옵션
- col_op : color option 포함 1, o.w. 0
> color options
> - WHITE, RED, GREEN, PINK, BLUE, ORANGE, YELLOW, PURPLE, BLACK, GREY, IVORY, BROWN
> - CHOC(CHOCOLATE), GOLD, SILVER, MINT
- 각 col_op에 대해 원-핫 인코딩
- des_op : 'design' 포함 1, o.w. 0

In [32]:
# 색 옵션에 대해 원-핫 인코딩
sales["temp"] = sales["Description"].str.extract("(WHITE|RED|GREEN|PINK|BLUE|ORANGE|YELLOW|PURPLE|BLACK|GREY|IVORY|BROWN|CHOC|GOLD|SILVER|MINT)")

oh_c = pd.get_dummies(sales["temp"])
sales = pd.concat([sales, oh_c], axis=1)

In [37]:
# Col_op 변수 추가
sales.loc[~sales["temp"].isna(), "temp"] = 1

In [38]:
label_encoder.fit(sales["temp"])
labels = label_encoder.transform(sales["temp"])
temp = pd.DataFrame(data=labels, columns=["Col_op"])
temp["Col_op"].replace({0:1, 1:0}, inplace=True)
sales["Col_op"] = temp["Col_op"]

In [57]:
# Des_op 변수 추가
sales["temp"] = sales["Description"].str.extract("(DESIGN)")

In [58]:
label_encoder.fit(sales["temp"])
labels = label_encoder.transform(sales["temp"])
temp = pd.DataFrame(data=labels, columns=["Des_op"])
temp["Des_op"].replace({0:1, 1:0}, inplace=True)
sales["Des_op"] = temp["Des_op"]

In [69]:
# 디자인 옵션에 대한 원-핫 인코딩 (중복 포함 상위 50개)
sales["temp"] = sales["Description"].str.extract("(SPACEBOY|SUKI|APPLE|ALPHABET|GIRL|VINTAGE|OWL|CERAMIC|BIRD|MIX|ASSTD|RETROSPOT|HEART|STRIP|BABUSHKA|GYMKHANA|SKETCHBOOK|COTTAGE|PAPER|CAT|PEAR|DOILEY|BALL|CLASSIC|SKULL|RABBIT|PAISLEY|BISCUIT|EMPIRE)")
oh_d1 = pd.get_dummies(sales["temp"])
sales = pd.concat([sales, oh_d1], axis=1)

sales["temp"] = sales["Description"].str.extract("(GIRL|LEAF|CAKE|DOG|CUT)")
oh_d2 = pd.get_dummies(sales["temp"])
sales = pd.concat([sales, oh_d2], axis=1)

### 상품 가격
- hprice : price가 일정 수준(5) 이상일 때 1, o.w. 0
- lprice : price가 일정 수준(1) 미만일 때 (음수 제외) 1, o.w. 0

In [71]:
# Hprice 변수 추가
strprice = sales["UnitPrice"].astype("str")
sales["temp"] = strprice.str.extract("([5-9]\.|[1-9]\d+\.)")
sales["temp"] = sales["temp"].astype("float")
sales.loc[~sales["temp"].isna(), "temp"] = 1

In [72]:
label_encoder.fit(sales["temp"])
labels = label_encoder.transform(sales["temp"])
temp = pd.DataFrame(data=labels, columns=["Hprice"])
temp["Hprice"].replace({0:1, 1:0}, inplace=True)
sales["Hprice"] = temp["Hprice"]

In [73]:
# Lprice 변수 추가
strprice = sales["UnitPrice"].astype("str")
sales["temp"] = strprice.str.extract("(0\.)")
sales["temp"] = sales["temp"].astype("float")
sales.loc[~sales["temp"].isna(), "temp"] = 1

In [74]:
label_encoder.fit(sales["temp"])
labels = label_encoder.transform(sales["temp"])
temp = pd.DataFrame(data=labels, columns=["Lprice"])
temp["Lprice"].replace({0:1, 1:0}, inplace=True)
sales["Lprice"] = temp["Lprice"]

### 상품 개수
- hquant : quantity가 일정 수준(10) 이상일 때 1, o.w. 0
- removed : quantity가 음수일 때 1, o.w. 0

In [76]:
# Hquant 변수 추가
strquant = sales["Quantity"].astype("str")
sales["temp"] = strquant.str.extract("([1-9]\d+)")
sales["temp"] = sales["temp"].astype("float")
sales.loc[~sales["temp"].isna(), "temp"] = 1

In [77]:
label_encoder.fit(sales["temp"])
labels = label_encoder.transform(sales["temp"])
temp = pd.DataFrame(data=labels, columns=["Hquant"])
temp["Hquant"].replace({0:1, 1:0}, inplace=True)
sales["Hquant"] = temp["Hquant"]

In [78]:
# Removed 변수 추가
strquant = sales["Quantity"].astype("str")
sales["temp"] = strquant.str.extract("(\-\d+)")
sales["temp"] = sales["temp"].astype("float")
sales.loc[~sales["temp"].isna(), "temp"] = 1

In [79]:
label_encoder.fit(sales["temp"])
labels = label_encoder.transform(sales["temp"])
temp = pd.DataFrame(data=labels, columns=["Removed"])
temp["Removed"].replace({0:1, 1:0}, inplace=True)
sales["Removed"] = temp["Removed"]

### 시계열 데이터
- quarter : 분기(Q1~4)에 따라 원-핫 인코딩
- year : 연도별 원-핫 인코딩
- am : 오전이면 1, o.w. 0

In [80]:
# Quarter 변수 추가 후 원-핫 인코딩
sales["InvoiceDate"] = pd.to_datetime(sales["InvoiceDate"])
sales["Quarter"] = sales["InvoiceDate"].dt.quarter
sales["Quarter"].replace({1:"Q1", 2:"Q2", 3:"Q3", 4:"Q4"}, inplace=True)

oh_q = pd.get_dummies(sales["Quarter"])
sales = pd.concat([sales, oh_q], axis=1)
sales.drop(columns=["Quarter"], inplace=True)

C:\Users\doram\AppData\Local\Temp\ipykernel_20240\1493849751.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales["Quarter"].replace({1:"Q1", 2:"Q2", 3:"Q3", 4:"Q4"}, inplace=True)


In [81]:
# Year 변수 추가 후 원-핫 인코딩
sales["temp"] = sales["InvoiceDate"].dt.year
oh_y = pd.get_dummies(sales["temp"])
df = pd.concat([sales, oh_y], axis=1)

In [82]:
# Am 변수 추가
sales["temp"] = sales["InvoiceDate"].dt.hour - 12
sales.loc[sales["temp"] >= 0, "temp"] = np.nan
sales.loc[sales["temp"] < 0, "temp"] = 1

In [83]:
label_encoder.fit(sales["temp"])
labels = label_encoder.transform(sales["temp"])
temp = pd.DataFrame(data=labels, columns=["Am"])
temp["Am"].replace({0:1, 1:0}, inplace=True)
sales["Am"] = temp["Am"]

### 국가
- country 변수를 이용해서 대륙별로 원-핫 인코딩
- 데이터 대부분이 UK이기 때문에 따로 분류

In [ ]:
# United Kingdom

# Europe : Germany, France, EIRE, Netherlands, Belgium, Switzerland, 
#          Norway, Channel Islands, Finland, Sweden, Austria, 
#          Denmark, Poland, Iceland, European Community, Lithuania, 
#          Czech Republic, Italy, Spain, Portugal, Greece, Cyprus, Malta

# Others : Unspecified, Australia, Japan, Israel, USA, Hong Kong, Singapore, Canada, 
#          United Arab Emirates, RSA, Lebanon, Brazil, Bahrain, Saudi Arabia

In [101]:
# 대륙별 원-핫 인코딩
sales["temp"] = 0
sales.loc[sales["Country"] == "United Kingdom", "temp"] = "United Kingdom"
sales.loc[(sales["Country"] == "Germany") | (sales["Country"] == "France") | (sales["Country"] == "EIRE") | (sales["Country"] == "Netherlands") | (sales["Country"] == "Belgium") | (sales["Country"] == "Switzerland") | (sales["Country"] == "Norway") | (sales["Country"] == "Channel Islands") | (sales["Country"] == "Finland") | (sales["Country"] == "Sweden") | (sales["Country"] == "Austria") | (sales["Country"] == "Denmark") | (sales["Country"] == "Poland") | (sales["Country"] == "Iceland") | (sales["Country"] == "European Community") | (sales["Country"] == "Lithuania") | (sales["Country"] == "Czech Republic") | (sales["Country"] == "Italy") | (sales["Country"] == "Spain") | (sales["Country"] == "Portugal") | (sales["Country"] == "Greece") | (sales["Country"] == "Cyprus") | (sales["Country"] == "Malta"), "temp"] = "Europe"
sales.loc[(sales["Country"] == "Unspecified") | (sales["Country"] == "Australia") | (sales["Country"] == "Japan") | (sales["Country"] == "Israel") | (sales["Country"] == "USA") | (sales["Country"] == "Hong Kong") | (sales["Country"] == "Singapore") | (sales["Country"] == "Canada") | (sales["Country"] == "United Arab Emirates") | (sales["Country"] == "RSA") | (sales["Country"] == "Lebanon") | (sales["Country"] == "Brazil") | (sales["Country"] == "Bahrain") | (sales["Country"] == "Saudi Arabia"), "temp"] = "Others"

oh_c = pd.get_dummies(sales["temp"])
sales = pd.concat([sales, oh_c], axis=1)

In [105]:
# 필요 없는 열 제거
sales.drop(columns=["index","temp"], inplace=True)

In [107]:
sales1 = sales.select_dtypes(include=[np.number])

In [108]:
sales1 = sales1.astype("int")

In [109]:
sales1

,Quantity,UnitPrice,y,Packed,Numbered,Set,Box,Amp,Plus,BLACK,...,Hquant,Removed,1,2,3,4,Am,Europe,Others,United Kingdom
0,6,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1
1,6,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1
2,8,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1
3,6,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1
4,6,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540450,12,0,0,1,1,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0
540451,6,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
540452,4,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
540453,4,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0


In [110]:
sales1.isna().sum().sum()

0

In [117]:
y_sales1 = sales1[['y']]
X_sales1 = sales1.drop('y',axis=1)

### 교차검증
- k=5인 교차검증을 진행

In [126]:
# micro
result_skfold = StratifiedKFold(n_splits=5)
result_clf = DecisionTreeClassifier(random_state=100)
n_iter = 0

cv_accuracy = []
cv_precision = []
cv_recall = []

for train_idx, test_idx in result_skfold.split(X_sales1, y_sales1):
    X_train, X_test = X_sales1.iloc[train_idx], X_sales1.iloc[test_idx]
    y_train, y_test = y_sales1.iloc[train_idx], y_sales1.iloc[test_idx]
    
    #학습을 진행
    result_clf.fit(X_train.values, y_train)
    
    #예측
    fold_pred = result_clf.predict(X_test.values)
    
    #정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, fold_pred), 3)
    precision = np.round(precision_score(y_test, fold_pred, average="micro"), 3)
    recall = np.round(recall_score(y_test, fold_pred, average="micro"), 3)
    print("\n{} 교차검증 정확도 :{}, 교차검증 정밀도 :{} 교차검증 재현율 :{}".format(n_iter, accuracy, precision, recall))
    
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    
print("\n")
print("\n 평균 검증 정확도", np.mean(cv_accuracy), np.mean(cv_precision), np.mean(cv_recall))


1 교차검증 정확도 :0.991, 교차검증 정밀도 :0.991 교차검증 재현율 :0.991

2 교차검증 정확도 :0.991, 교차검증 정밀도 :0.991 교차검증 재현율 :0.991

3 교차검증 정확도 :0.99, 교차검증 정밀도 :0.99 교차검증 재현율 :0.99

4 교차검증 정확도 :0.989, 교차검증 정밀도 :0.989 교차검증 재현율 :0.989

5 교차검증 정확도 :0.992, 교차검증 정밀도 :0.992 교차검증 재현율 :0.992



 평균 검증 정확도 0.9905999999999999 0.9905999999999999 0.9905999999999999


In [127]:
# macro
result_skfold = StratifiedKFold(n_splits=5)
result_clf = DecisionTreeClassifier(random_state=100)
n_iter = 0

cv_accuracy = []
cv_precision = []
cv_recall = []

for train_idx, test_idx in result_skfold.split(X_sales1, y_sales1):
    X_train, X_test = X_sales1.iloc[train_idx], X_sales1.iloc[test_idx]
    y_train, y_test = y_sales1.iloc[train_idx], y_sales1.iloc[test_idx]
    
    #학습을 진행
    result_clf.fit(X_train.values, y_train)
    
    #예측
    fold_pred = result_clf.predict(X_test.values)
    
    #정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, fold_pred), 3)
    precision = np.round(precision_score(y_test, fold_pred, average="macro"), 3)
    recall = np.round(recall_score(y_test, fold_pred, average="macro"), 3)
    print("\n{} 교차검증 정확도 :{}, 교차검증 정밀도 :{} 교차검증 재현율 :{}".format(n_iter, accuracy, precision, recall))
    
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    
print("\n")
print("\n 평균 검증 정확도", np.mean(cv_accuracy), np.mean(cv_precision), np.mean(cv_recall))


1 교차검증 정확도 :0.991, 교차검증 정밀도 :0.732 교차검증 재현율 :0.607

2 교차검증 정확도 :0.991, 교차검증 정밀도 :0.756 교차검증 재현율 :0.781

3 교차검증 정확도 :0.99, 교차검증 정밀도 :0.689 교차검증 재현율 :0.741

4 교차검증 정확도 :0.989, 교차검증 정밀도 :0.707 교차검증 재현율 :0.553

5 교차검증 정확도 :0.992, 교차검증 정밀도 :0.928 교차검증 재현율 :0.635



 평균 검증 정확도 0.9905999999999999 0.7624 0.6634


In [128]:
# weighted
result_skfold = StratifiedKFold(n_splits=5)
result_clf = DecisionTreeClassifier(random_state=100)
n_iter = 0

cv_accuracy = []
cv_precision = []
cv_recall = []

for train_idx, test_idx in result_skfold.split(X_sales1, y_sales1):
    X_train, X_test = X_sales1.iloc[train_idx], X_sales1.iloc[test_idx]
    y_train, y_test = y_sales1.iloc[train_idx], y_sales1.iloc[test_idx]
    
    #학습을 진행
    result_clf.fit(X_train.values, y_train)
    
    #예측
    fold_pred = result_clf.predict(X_test.values)
    
    #정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, fold_pred), 3)
    precision = np.round(precision_score(y_test, fold_pred, average="weighted"), 3)
    recall = np.round(recall_score(y_test, fold_pred, average="weighted"), 3)
    print("\n{} 교차검증 정확도 :{}, 교차검증 정밀도 :{} 교차검증 재현율 :{}".format(n_iter, accuracy, precision, recall))
    
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    
print("\n")
print("\n 평균 검증 정확도", np.mean(cv_accuracy), np.mean(cv_precision), np.mean(cv_recall))


1 교차검증 정확도 :0.991, 교차검증 정밀도 :0.988 교차검증 재현율 :0.991

2 교차검증 정확도 :0.991, 교차검증 정밀도 :0.992 교차검증 재현율 :0.991

3 교차검증 정확도 :0.99, 교차검증 정밀도 :0.99 교차검증 재현율 :0.99

4 교차검증 정확도 :0.989, 교차검증 정밀도 :0.986 교차검증 재현율 :0.989

5 교차검증 정확도 :0.992, 교차검증 정밀도 :0.992 교차검증 재현율 :0.992



 평균 검증 정확도 0.9905999999999999 0.9895999999999999 0.9905999999999999


- micro : 정확도, 정밀도, 재현율 모두 평균적으로 아주 높게 (>0.99) 나왔다.
- macro : 정확도는 아주 높게 나왔지만, 정밀도는 평균적으로 약 0.76, 재현율은 약 0.66으로 낮게 나왔다.
- weighted : 정확도, 정밀도, 재현율 모두 아주 높게 나왔다. 정밀도의 경우도 아주 높지만 정확도와 재현율에 비해 미세하게 낮게 나왔다.
- macro average가 낮게 나온 것은 레이블별 데이터의 수의 차이가 원인인 것으로 보인다.